In [1]:
from lxml import html
import lxml
import time
import requests
import pandas as pd
import sqlite3
import re

In [2]:
date_jour = '2017-12-31'
OUTPUT_PATH = '../_outputs/'
database_file = OUTPUT_PATH + 'chefs_{}.db'.format(date_jour)
conn = sqlite3.connect(database_file)

In [3]:
df_source = pd.read_sql_query("select * from index_chefs;", conn)
df_source.head()

,index,chef,total
0,0,florent-gouverneur,14
1,1,arnaud-bulliot,16
2,2,quentin-girerd,2
3,3,dario-jarel,12
4,4,la-belle-assiette-france,540


In [4]:
def getChefPage(chef_id):
    url = 'https://labelleassiette.fr/' + chef_id
    response = requests.get(url)
    content_source = response.text
    return [chef_id, html.document_fromstring(content_source)]

In [5]:
docs = []
for index, row in df_source.iterrows():
    docs.append(getChefPage(row['chef']))

Page : https://labelleassiette.fr/florent-gouverneur
Page : https://labelleassiette.fr/arnaud-bulliot
Page : https://labelleassiette.fr/quentin-girerd
Page : https://labelleassiette.fr/dario-jarel
Page : https://labelleassiette.fr/la-belle-assiette-france
Page : https://labelleassiette.fr/kevin-bense
Page : https://labelleassiette.fr/sacharanin-son-herlidou
Page : https://labelleassiette.fr/daniel-lopez
Page : https://labelleassiette.fr/raphael-courtin
Page : https://labelleassiette.fr/ella-aflalo
Page : https://labelleassiette.fr/mariana-delubac
Page : https://labelleassiette.fr/mickael-houx
Page : https://labelleassiette.fr/pierre-guichon
Page : https://labelleassiette.fr/julie-anne-charton
Page : https://labelleassiette.fr/cedric-fortes
Page : https://labelleassiette.fr/antoine-collin
Page : https://labelleassiette.fr/mojgan-tashvighi
Page : https://labelleassiette.fr/luc-fischer
Page : https://labelleassiette.fr/thomas-major
Page : https://labelleassiette.fr/hina-malik
Page : https

Page : https://labelleassiette.fr/stephane-oulevey
Page : https://labelleassiette.fr/eleonor-ill-walgener
Page : https://labelleassiette.fr/elise-deat
Page : https://labelleassiette.fr/herve-escobar
Page : https://labelleassiette.fr/sombat-et-gregor
Page : https://labelleassiette.fr/zelikha-dinga
Page : https://labelleassiette.fr/eleonora-galasso
Page : https://labelleassiette.fr/tomoko-ogata
Page : https://labelleassiette.fr/hanyen-hsu
Page : https://labelleassiette.fr/yuan-ren
Page : https://labelleassiette.fr/noemi-ito
Page : https://labelleassiette.fr/ji-hye-park
Page : https://labelleassiette.fr/mattia-salvadore
Page : https://labelleassiette.fr/jonathan-lefevre
Page : https://labelleassiette.fr/aurelie-scarabin
Page : https://labelleassiette.fr/fanny-matuszkiewicz
Page : https://labelleassiette.fr/aurelie-gauvin
Page : https://labelleassiette.fr/laura-celine-you
Page : https://labelleassiette.fr/boris-banicles
Page : https://labelleassiette.fr/arthur-lefranc
Page : https://labell

In [6]:
len(docs)

284

In [26]:
docs[0][1]

<Element html at 0x1189d49f8>

In [88]:
# https://stackoverflow.com/questions/6937525/escaping-xpath-literal-with-python
def toXPathStringLiteral(s):
    if "'" not in s: return "'%s'" % s
    if '"' not in s: return '"%s"' % s
    return "concat('%s')" % s.replace("'", "',\"'\",'")

In [89]:
def get_rows(doc):
    
    menus = doc.xpath('//h3[@class="menu-title"]/text()')
    
    descriptions = doc.xpath('//p[@class="menu-resume extract"]/text()')
    
    tags = []
    descriptions = []
    
    for m in menus:
        if(m):
            tags.append(
                doc.xpath(
                    '//h3[text()=%s]/following-sibling::ul[@class="tags"]/li/text()' 
                    % toXPathStringLiteral(m)
                )
            )
            
            descriptions.append(
                doc.xpath(
                    '//h3[text()=%s]/../following-sibling::div[@class="menu-details full"]//p[@class="dish"]/text()' 
                    % toXPathStringLiteral(m)
                )
            )
        
        else:
            tags.append('NA')
    
    scripts = doc.findall('.//script')
    prices = []
    item = 0
    for s in scripts:
        if 'chef.menus[{}]'.format(item) in s.text_content():
            prices.append(s.text_content())
            item += 1  
    # print(prices[1])
    return menus, descriptions, tags, prices

In [90]:
columns = ['chef', 'menu', 'description', 'tags', 'prices']
df_output = pd.DataFrame(columns=columns)

In [92]:
idx = 0
for doc in docs:
    menus, descriptions, tags, prices = get_rows(doc[1])
    chef = doc[0]
    # print(chef)
    idxm = 0
    for m in menus:
        # print(m)
        df_output.loc[idx] = [chef, m, descriptions[idxm], tags[idxm], prices[idxm]]
        idx +=1
        idxm +=1

In [93]:
df_output.head(50)

,chef,menu,description,tags,prices
0,florent-gouverneur,Cocktail Dinatoire,"[\nMini burgers\n, \nMini Croque-Monsieur\n, \...","[Moderne, Français, Bio, Végétalien, Poisson, ...","chef.menus[0]={prices:[],defaultPrice:'0'};che..."
1,florent-gouverneur,Du Vert Au Vers,[\nCrémeux potiron aux éclats de noisettes et ...,[],"chef.menus[1]={prices:[],defaultPrice:'0'};che..."
2,florent-gouverneur,Menu Découverte,"[\nTartare de crevettes et gelée d'agrumes\n, ...","[Moderne, Français, Bio, Sans Gluten, Poisson,...","chef.menus[2]={prices:[],defaultPrice:'0'};che..."
3,florent-gouverneur,Menu Herbacé,"[\nFilet de maquereau en escabèche, et coulis ...","[Français, Automne, Moderne]","chef.menus[3]={prices:[],defaultPrice:'0'};che..."
4,florent-gouverneur,Menu Automnal,[\nMéli-mélo de choux-fleurs orange girolles e...,"[Sans Gluten, Végétarien, Poisson, Viande, Fra...","chef.menus[4]={prices:[],defaultPrice:'0'};che..."
5,florent-gouverneur,Bistro...nomique,"[\nVichyssoise et chips de lard fumé\n, \nAgne...","[Viande, Français, Automne, Hiver, Bistronomiq...","chef.menus[5]={prices:[],defaultPrice:'0'};che..."
6,florent-gouverneur,Veggies D'automne,[\nRavioles de champignons dans son bouillon\n...,"[Bistronomique, Moderne, Français, Végétarien,...","chef.menus[6]={prices:[],defaultPrice:'0'};che..."
7,arnaud-bulliot,Saveur De Nos Montagnes,"[\nTartine de chèvre frais, miel et jambon cru...","[Pescaterien, Poisson, Viande, Français, Autom...","chef.menus[0]={prices:[],defaultPrice:'0'};che..."
8,arnaud-bulliot,Douceur Hivernale,"[\nSelon mon humeur !\n, \nBarigoule d'articha...","[Poisson, Viande, Français, Hiver, Traditionne...","chef.menus[1]={prices:[],defaultPrice:'0'};che..."
9,arnaud-bulliot,Saveur De Nos Bistrots,"[\nSamossa de chèvre, miel et menthe fraîche\n...","[Pescaterien, Poisson, Viande, Français, Print...","chef.menus[2]={prices:[],defaultPrice:'0'};che..."


In [94]:
df_output.to_excel(OUTPUT_PATH + 'chefs_{}_menus.xlsx'.format(date_jour), index=False, header=True)